<a href="https://colab.research.google.com/github/jufabeck2202/KI-Lab/blob/main/%5BKI_Lab2021%5D_Assignmanet_5_OWCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 5: 
Text Preparation:
1. We use the same dataset from last time: https://github.com/tblock/10kGNAD.
2. Download the files train.csv and test.csv
3.Filter both files to include only lines that fulfill both of the following conditions:
    1. The line is of the category „Wirtschaft“ or „Sport“.
    2. The line only contains valid characters (note that ‘\‘ is a line break in Python):
4. The number of valid lines for training and test should be 2154 and 244.

5. Divide the valid training data lines into tokens using SpaCy.

---------------------------------------------

6. Determine the 5000 most common tokens in the training data.
7. For those tokens, create the word_to_ix dictionary as shown in the tutorial.
8. Train a language classification model using the two categories and the sentences using an LSTM model

9. Use the same architecture as in the tutorial but an embedding dim of 128 and a hidden dim of 256.
10. In one training loop go through every line of the training data and update the weights after every line (like in the tutorial).
11. For every line, transform the words according to the word_to_ix dictionary and feed it to the LSTM as one sequence. Note: Skip words that are not in word_to_ix (they are not part of the 5000 most common tokens.)
12. Do this for 5 epochs and print the accuracy on the test data after every epoch.
13. Note: Since we do not train in batches, but line per line, this will take quite a while (~ 15 minutes for one training loop).

#Bonus
14. The training is quite slow, since we do not train in batches but line by line. Providing the text lines in batches to the LSTM, will not work out of the box, because all tensors in one batch need to have the same length.
15. The DataLoader class can be created with a collate_fn function that can be used to bring all lines in one batch to the same size:
16. Determine the longest tensor in the current batch.
17. Fill up all tensors that are shorter with an unused padding value (e.g. -1).
18. Task: Put the data into a DataLoader, pad the lines in one batch to the same length and train in batches on the GPU. Compare the training time to the line-wise training.

#Import

In [30]:
!pip install -U spacy[cuda100] de &> /dev/null
!python -m spacy download de &> /dev/null
!pip install -q "tqdm>=4.36.1" &> /dev/null

In [80]:
import pandas as pd
import torch
import spacy
spacy.prefer_gpu()
nlp_ger = spacy.load("de")
from tqdm import tqdm
import string
import torch
import torch.nn as nn
import random
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from collections import Counter
from tabulate import tabulate

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


# Import files


In [32]:
df_test = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/test.csv', \
                      names=['trash','category','Data'], sep='^([^;]+);', engine="python")
df_test = df_test.drop(columns=['trash'])
df_test

,category,Data
0,Wirtschaft,"'Die Gewerkschaft GPA-djp lanciert den ""All-in..."
1,Sport,Franzosen verteidigen 2:1-Führung – Kritische ...
2,Web,'Neues Video von Designern macht im Netz die R...
3,Sport,23-jähriger Brasilianer muss vier Spiele pausi...
4,International,Aufständische verwendeten Chemikalie bei Gefec...
...,...,...
1023,Web,"Programm ""Ebay Plus"" soll Ende September start..."
1024,Inland,"Fehlerhafte Bilanz 2013, Partei will berufen –..."
1025,Sport,Als erster Olympia-Champion auch Hawaii-Sieger...
1026,Sport,Der Außenseiter aus Denver gewinnt die 50. Sup...


In [33]:
df_train = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv', \
                      names=['trash','category','Data'], sep='^([^;]+);', engine="python")
df_train = df_train.drop(columns=['trash'])

df_train

,category,Data
0,Sport,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,Kultur,"'Erfundene Bilder zu Filmen, die als verloren ..."
2,Web,Der frischgekürte CEO Sundar Pichai setzt auf ...
3,Wirtschaft,"Putin: ""Einigung, dass wir Menge auf Niveau vo..."
4,Inland,Estland sieht den künftigen österreichischen P...
...,...,...
9240,Inland,Bernd Saurer war Bridge-Juniorenweltmeister un...
9241,International,Sandhere soll in vergangener Woche bei Luftang...
9242,Wirtschaft,Derzeit Konzeptgruppe in Berlin – Kein Komment...
9243,Inland,Landeshauptmann will den vierten Regierungssit...


In [34]:
#filter:
df_test = df_test.loc[(df_test["category"]=="Wirtschaft") | (df_test["category"]=="Sport")]
df_train = df_train.loc[(df_train["category"]=="Wirtschaft") | (df_train["category"]=="Sport")]

print(len(df_test))
print(len(df_train))

261
2351


In [35]:
rows_test = []
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0], position=0, leave=True):
    rows_test.append(row.Data)

rows_train = []
for index, row in tqdm(df_train.iterrows(), total=df_train.shape[0], position=0, leave=True):
    rows_train.append(row.Data)


100%|██████████| 2351/2351 [00:00<00:00, 11890.76it/s]


In [38]:
all_letters = string.ascii_letters + 'ÄÖÜäöüß–' + string.punctuation + \
                string.digits + string.whitespace

rows_test_1 = [line for line in rows_test if all(ch in list(all_letters) for ch in line)]
print(len(rows_test_1))

rows_train_1 = [line for line in rows_train if all(ch in list(all_letters) for ch in line)]
print(len(rows_train_1))

261
244
2351
2154


In [85]:
rows_train_token = []
for sentence in tqdm(rows_train_1, position=0, leave=True):
    token = nlp_ger(sentence)
    for token_s in token:
        rows_train_token.append(token_s.text)

100%|██████████| 2154/2154 [01:30<00:00, 23.83it/s]


In [86]:
word_freq = Counter(rows_train_token)
common_words = word_freq.most_common(5000)
print(common_words)

[('.', 45468), (',', 40543), ('der', 22639), ('die', 19536), ('in', 13411), ('und', 12308), ('(', 9056), ('den', 8954), (')', 8914), ('von', 7050), ('mit', 6765), ('–', 6456), ('im', 6280), ('auf', 6082), ('das', 5982), ('zu', 5699), (':', 5650), ('für', 5497), ('ist', 5447), ('Die', 5371), ('nicht', 5366), ('sich', 4838), ('dem', 4559), ('ein', 4363), ('es', 4280), (' ', 4049), ('am', 3868), ('des', 3752), ('eine', 3727), ('auch', 3704), ('nach', 3453), ('bei', 3393), ('als', 3367), ('Der', 3350), ('hat', 3176), ('an', 3002), ('dass', 2954), ('aus', 2710), ('war', 2606), ('aber', 2550), ('werden', 2490), ('einem', 2479), ('noch', 2426), ('gegen', 2413), ('um', 2408), ('vor', 2356), ('Das', 2327), ('einen', 2254), ('haben', 2204), ('Prozent', 2176), ('wird', 2169), ('sind', 2109), ('er', 2052), ('Euro', 2049), ('wie', 2046), ('einer', 2001), ('nur', 1868), ('mehr', 1840), ('über', 1727), ('hatte', 1677), ('zum', 1629), ('sie', 1591), ('/', 1493), ('so', 1475), ('In', 1469), ('man', 137

In [ ]:
word_to_ix = {}
for sent in rows_train_token:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)